In [1]:
import os, glob
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from zipfile import ZipFile
import pdb

In [2]:
# get data location
#base_path = "/Users/nikkibytes/Documents/niblunc/BCP/BCP_DATA"#"/Users/nikkibytes/Documents/BCP_DATA"
basepath = "/Users/gracer/Google Drive/BCP/data"
#text4_files=glob.glob(os.path.join(base_path, "set*/*/*/*04.txt"))
#text9_files=glob.glob(os.path.join(base_path, "set*/*/*/*09.txt"))

# Read in the data from the zip files

In [3]:
list_of_files = {}
for (dirpath, dirnames, filenames) in os.walk(basepath):
    for filename in filenames:
        if filename.endswith('.zip'):
            #print(filename)
            tmppath=os.sep.join([dirpath, filename])
            #print(tmppath)
            with ZipFile(tmppath, 'r') as zipObj:
               # Get a list of all archived file names from the zip
               listOfFileNames = zipObj.namelist()
               # Iterate over the file names
               for fileName in listOfFileNames:
                   # Check filename endswith csv
                    if fileName.endswith('04.txt'):
                                           # Extract a single file from zip
                        zipObj.extract(fileName, os.path.join(basepath,'temp_txt'))
                    if fileName.endswith('09.txt'):
                                           # Extract a single file from zip
                        zipObj.extract(fileName, os.path.join(basepath,'temp_txt'))
infile = os.path.join(basepath,'temp_txt')

# Dictionary of the files we want

In [4]:
file_dict = {"set_04": {"mom":{}, "child": {}}, "set_09": {"mom": {}, "child": {}}}
file_dict['set_04']["mom"]["files"] = [x for x in glob.glob(os.path.join(infile,'*04.txt')) if "Mom" in x]
file_dict['set_09']["mom"]["files"] = [x for x in glob.glob(os.path.join(infile,'*09.txt')) if "Mom" in x]
file_dict['set_04']["child"]["files"] = [x for x in glob.glob(os.path.join(infile,'*04.txt')) if "Chil" in x]
file_dict['set_09']["child"]["files"] = [x for x in glob.glob(os.path.join(infile,'*09.txt')) if "Chil" in x]


# Concat

In [5]:
temp_list = []            
for file in file_dict["set_04"]["mom"]["files"]:
    temp_df =  pd.read_csv(file, sep="\t", encoding='latin1')
    temp_df=temp_df.drop([0])
    for val in temp_df["Participant ID"]:
        _id = str(val).lstrip("0").split("_")[0]
        temp_df.replace(val, _id, inplace=True)
    temp_list.append(temp_df)
dfm4_original = pd.concat(temp_list, ignore_index=True)
print("Final dataframe size: ", dfm4_original.shape)


dfm4_orignal = dfm4_original.sort_values(by="Participant ID")

Final dataframe size:  (360, 226)


In [6]:
temp_list = []            
for file in file_dict["set_09"]["mom"]["files"]:
    temp_df = pd.read_csv(file,encoding='latin1', sep="\t")
    temp_df=temp_df.drop([0])
    for val in temp_df["Participant ID"]:
        _id = str(val).strip("0").strip(".").split("_")[0]
        temp_df.replace(val, _id, inplace=True)
    temp_list.append(temp_df)
dfm9_original = pd.concat(temp_list, ignore_index=True)
dfm9_orignal = dfm9_original.sort_values(by="Participant ID")

# Saving

In [7]:
concat_filepath = os.path.join(basepath,'Concat','Mom_BCP_dataset4.txt')
dfm4_original.to_csv(concat_filepath, index=False, sep="\t", header=True)

In [8]:
concat_filepath = os.path.join(basepath,'Concat','Mom_BCP_dataset9.txt')
dfm9_original.to_csv(concat_filepath, index=False, sep="\t", header=True)

# Remove stupid fucking commas

In [9]:
for col in dfm4_original:
    if dfm4_original[col].dtype == np.object_:
        dfm4_original[col] = (dfm4_original[col].replace(',','.', regex=True))
        

# Join 04 and 09

## Find common elements

In [10]:
b=list(dfm9_original['Participant ID'])

In [11]:
a=list(dfm4_original['Participant ID'])

In [12]:
common=list(set(a) & set(b))

## Remove partcipants who are missing either 04 or 09

In [13]:
missmatch = list(set(a)-set(b))

In [14]:
common4=dfm4_original[dfm4_original['Participant ID'].isin(common)]

In [15]:
common9=dfm9_original[dfm9_original['Participant ID'].isin(common)]

In [16]:
total_df = common4.merge(common9.drop_duplicates(subset=['Project Abbreviation','Date of Intake']), how='left')

In [17]:
total_df=total_df.dropna(axis=1, how='all')

In [18]:
important=['Participant ID','VEG0100','VEG0200','VEG0300','VEG0400','VEG0800','VEG0450','VEG0700',
           'VEG0600','VEG0900','VEG0500','VEG0100','VEG0700','FRU0100','FRU0200','FRU0300','FRU0400',
           'FRU0500','FRU0600','FRU0700','FRU0300','FRU0400','FRU0500','FRU0600','FRU0700',
           'Whole Grains (ounce equivalents)','DMF0100','DMR0100','DML0100','DMN0100','DMF0200',
           'DMR0200','DML0200','DML0300','DML0400','DCF0100','DCR0100','DCL0100','DCN0100','DYF0100',
           'DYR0100','DYL0100','DYF0200','DYR0200','DYL0200','DYN0100','DOT0300','DOT0400','DOT0500',
           'DOT0600','DOT0100','MRF0100','MRL0100','MRF0200','MRL0200','MRF0300','MRL0300','MRF0400',
           'MRL0400','MCF0200','MCL0200','MRF0500','MPF0100','MPL0100','MPF0200','MFF0100','MFL0100',
           'MFF0200','MSL0100','MSF0100','MCF0100','MCL0100','MOF0100','MOF0200','MOF0300','MOF0400',
           'MOF0500','MOF0600','MOF0700','VEG0700','MFF0100','MFL0100','MFF0200','MSL0100','MSF0100',
           'MOF0500','MOF0600','MOF0700','VEG0700','Sodium (mg)','Refined Grains (ounce equivalents)',
           'Added Sugars (by Total Sugars) (g)','% Calories from SFA','Energy (kcal)',
           'Total Polyunsaturated Fatty Acids (PUFA) (g)','Total Monounsaturated Fatty Acids (MUFA) (g)',
           'Total Saturated Fatty Acids (SFA) (g)']

In [19]:
complete=total_df[total_df.columns.intersection(important)].dropna()

In [20]:
cind=list(complete.index)

In [21]:
complete_df=total_df[total_df.index.isin(cind)]

In [22]:
missing_df=total_df[~total_df.index.isin(cind)]

In [48]:
mm4=dfm4_original[dfm4_original['Participant ID'].isin(missmatch)]
mm9=dfm9_original[dfm9_original['Participant ID'].isin(missmatch)]

In [50]:
missing_df=missing_df.append(mm4)

In [51]:
missing_df=missing_df.append(mm9)

In [23]:
concat_filepath = os.path.join(basepath,'Concat','Mom_BCP_datasetTOTAL.txt')
complete_df.to_csv(concat_filepath, index=False, sep="\t", header=True)

# Parameter dictionary

In [24]:
para_dict = {'hei_totveg': {'parameters':[1.1], 'name': 'HEIX1_TOTALVEG'}, 
             'hei_greensbeans': {'parameters':[0.2], 'name': 'HEIX2_GREEN_AND_BEAN'},
             'hei_totfruit': {'parameters':[0.8], 'name': 'HEIX3_TOTALFRUIT'},
             'hei_wholefruit': {'parameters':[0.4], 'name': 'HEIX4_WHOLEFRUIT'},
             'hei_wholegrains': {'parameters':[1.5], 'name': 'HEIX5_WHOLEGRAIN'},             
             'hei_dairy': {'parameters':[1.3], 'name': 'HEIX6_TOTALDAIRY'},
             'hei_totproteins': {'parameters':[2.5], 'name': 'HEIX7_TOTPROT'},
             'hei_seafoodplantprot': {'parameters':[0.8], 'name': 'HEIX8_SEAPLANT_PROT'},
             'hei_refinedgrains': {'parameters':[1.8,4.3], 'name': 'HEIX11_REFINEDGRAIN'},
             'hei_addedsugars': {'parameters':[6.5,26], 'name': 'HEIX12_ADDEDSUGARS'},
             'hei_SFA': {'parameters':[8,16], 'name': 'HEIX13_SATFATS'},
             'Fats': {'parameters':[1.2,2.5], 'name': 'HEIX9_FATTYACID'},
             'hei_sodium':{'parameters':[1.1,2.0],'name':'HEIX10_SODIUM'}
            }


In [25]:
hei_dict={'hei_totveg':
          ['VEG0100','VEG0200','VEG0300','VEG0400','VEG0800','VEG0450','VEG0700','VEG0600','VEG0900','VEG0500'],
          'hei_greensbeans':
          ['VEG0100','VEG0700'],
          'hei_totfruit':
          ['FRU0100','FRU0200','FRU0300','FRU0400','FRU0500','FRU0600','FRU0700'],
          'hei_wholefruit':
          ['FRU0300','FRU0400','FRU0500','FRU0600','FRU0700'],
          'hei_wholegrains':
          ['Whole Grains (ounce equivalents)'],
          'hei_dairy':
          ['DMF0100','DMR0100','DML0100','DMN0100','DMF0200','DMR0200','DML0200',
                       'DML0300','DML0400','DCF0100','DCR0100','DCL0100','DCN0100','DYF0100',
                       'DYR0100','DYL0100','DYF0200','DYR0200','DYL0200','DYN0100',
                       'DOT0300','DOT0400','DOT0500','DOT0600','DOT0100'],
          'hei_totproteins':
          ['MRF0100','MRL0100','MRF0200','MRL0200','MRF0300','MRL0300','MRF0400',
                             'MRL0400','MCF0200','MCL0200','MRF0500','MPF0100','MPL0100','MPF0200',
                             'MFF0100','MFL0100','MFF0200','MSL0100',
                             'MSF0100','MCF0100','MCL0100','MOF0100','MOF0200','MOF0300','MOF0400','MOF0500',
                             'MOF0600','MOF0700','VEG0700'],
          'hei_seafoodplantprot':
          ['MFF0100','MFL0100','MFF0200','MSL0100','MSF0100','MOF0500','MOF0600','MOF0700','VEG0700'],
          'hei_sodium':
          ['Sodium (mg)'],
          'hei_refinedgrains':
          ['Refined Grains (ounce equivalents)'],
          'hei_addedsugars':
          ['Added Sugars (by Total Sugars) (g)'], 
          'ripctsfa': ['% Calories from SFA','Energy (kcal)'],
         'energy':
         ['Energy (kcal)'],
         'fats':
         ['Total Polyunsaturated Fatty Acids (PUFA) (g)','Total Monounsaturated Fatty Acids (MUFA) (g)',
         'Total Saturated Fatty Acids (SFA) (g)']}

# Create the hei variables

In [26]:
for key, value in hei_dict.items():
    print(key)
    if key in ['hei_totveg','hei_greensbeans','hei_totfruit', 'hei_wholefruit']:
        x=value
        complete_df[key] = complete_df[x].astype('float').sum(axis=1)
        complete_df[key] = complete_df[key]/2
    if key in ['hei_wholegrains','hei_refinedgrains']:
        x=value
        complete_df[key] = complete_df[x].astype('float')
    if key in ['hei_dairy']:
        x=value[:-1]
        tmp= complete_df[x].astype('float').sum(axis=1)
        y=value[-1]
        if y == 'DOT0100':
            tmp2=complete_df[y].astype('float')/3
            complete_df[key]=tmp+tmp2
        else:
            print('NO DAIRY MISSING DOT0100, needs to be last in list')
    if key in ['hei_totproteins']:
        x=value[:-1]
        tmp= complete_df[x].astype('float').sum(axis=1)
        y=value[-1]
        if y == 'VEG0700':
            tmp2=complete_df[y].astype('float')*2
            complete_df[key]=tmp+tmp2
        else:
            print('NO TOTAL PROTEIN MISSING VEG0700, needs to be last in list')
    if key in ['hei_seafoodplantprot']:
        x=value[:-1]
        tmp= complete_df[x].astype('float').sum(axis=1)
        y=value[-1]
        if y == 'VEG0700':
            tmp2=complete_df[y].astype('float')*2
            complete_df[key]=tmp+tmp2
        else:
            print('NO SEAFOOD AND PLANT PROTEIN MISSING VEG0700, needs to be last in list')
    if key in ['hei_sodium']:
        x=value
        complete_df[key] = complete_df[x].astype('float')/1000
    if key in ['hei_addedsugars']:
        x=value
        complete_df[key] = complete_df[x].astype('float')*4
    if key in ['ripctsfa']:
        x=value[0]
        y=value[1]
        complete_df[key] = complete_df[x].astype('float')*complete_df[y].astype('float')
    if key in ['energy']:
        x=value
        complete_df[key] = complete_df[x].astype('float')/1000

hei_totveg
hei_greensbeans
hei_totfruit
hei_wholefruit
hei_wholegrains
hei_dairy
hei_totproteins
hei_seafoodplantprot
hei_sodium
hei_refinedgrains
hei_addedsugars
ripctsfa
energy
fats


proc means data=hei0409togroup noprint;
  by cpartid;
  var rikcal hei_totveg hei_greensbeans hei_totfruit hei_wholefruit hei_wholegrains hei_dairy hei_totproteins hei_seafoodplantprot 
		ripfa rimfa risfa hei_sodium hei_refinedgrains hei_addedsugars ripctsfa;
  output out=dailyhei0409 sum=;
run;

# Grouping

In [27]:
interest = ['Participant ID','Energy (kcal)', 'hei_totveg', 'hei_greensbeans', 'hei_totfruit', 'hei_wholefruit', 'hei_wholegrains',
            'hei_dairy', 'hei_totproteins', 'hei_seafoodplantprot', 'Total Polyunsaturated Fatty Acids (PUFA) (g)',
            'Total Monounsaturated Fatty Acids (MUFA) (g)', 'Total Saturated Fatty Acids (SFA) (g)', 
            'hei_sodium', 'hei_refinedgrains', 'hei_addedsugars', 'ripctsfa','energy','% Calories from SFA']

In [28]:
dailyhei0409=complete_df[complete_df.columns.intersection(interest)].groupby(['Participant ID']).mean()

In [29]:
data_dict={}

In [30]:
data_dict['hei0409']=complete_df
data_dict['dailyhei0409']=dailyhei0409

# Important functions

In [31]:
def adeq_check(df,inputt, output, parameter):
    if inputt in ['hei_totveg','hei_greensbeans', 'hei_totfruit', 'hei_wholefruit', 'hei_totproteins', 'hei_seafoodplantprot']:
        tmp = df[inputt]/df['energy']
        df[output] = [5 if x >= parameter else 5*(x/parameter) for x in tmp]
    elif inputt in ['hei_wholegrains', 'hei_dairy']:
        if inputt == 'hei_wholegrains':
            tmp = df[inputt]/df['energy']
            df[output] = [10 if x >= parameter else 10*(x/parameter) for x in tmp]
        else:
            tmp = df[inputt]/df['energy']
            df[output] = [10 if x >= parameter else 10*(x/parameter) for x in tmp]
    elif inputt in ['Fats']:
        FARMIN=parameter[0]
        FARMAX=parameter[1]
        tmp=df['Total Polyunsaturated Fatty Acids (PUFA) (g)']+df['Total Monounsaturated Fatty Acids (MUFA) (g)']
        tmp2=tmp/df['Total Saturated Fatty Acids (SFA) (g)']
        df[output] = [10 if x > FARMAX else 0 if x <= FARMIN else 10*((x-FARMIN)/(FARMAX-FARMIN)) for x in tmp2]  


In [32]:
def mod_check(df,inputt, output, parameter):    
    if inputt in ['hei_sodium','hei_refinedgrains']:
        tmp = df[inputt]/df['energy']
        df[output] = [0 if x >= parameter[1] else 10 if x <= parameter[0] else 10-(10*((x-parameter[0])/(parameter[1]-parameter[0]))) for x in tmp]
    if inputt in ['hei_addedsugars']:
        tmp= 100*df[inputt]/df['energy']
        df[output] = [0 if x >= parameter[1] else 10 if x < parameter[0] else 10-(10*((x-parameter[0])/(parameter[1]-parameter[0]))) for x in tmp]
    if inputt in ['hei_SFA']:
        tmp= df['% Calories from SFA']
        df[output] = [0 if x > parameter[1] else 10 if x < parameter[0] else 10-(10*((x-parameter[0])/(parameter[1]-parameter[0]))) for x in tmp]
        

In [44]:
def check(x, data):
    df=data
    toSum=['HEIX1_TOTALVEG','HEIX2_GREEN_AND_BEAN' , 'HEIX3_TOTALFRUIT' , 'HEIX4_WHOLEFRUIT' , 
           'HEIX5_WHOLEGRAIN' , 'HEIX6_TOTALDAIRY' , 'HEIX7_TOTPROT' , 'HEIX8_SEAPLANT_PROT' , 'HEIX9_FATTYACID' , 
           'HEIX10_SODIUM' , 'HEIX11_REFINEDGRAIN' , 'HEIX12_ADDEDSUGARS' , 'HEIX13_SATFATS']
    for key,values in x.items():
        print('Calculating score for %s'%key)
        if key in ['hei_totveg','hei_greensbeans', 'hei_totfruit', 'hei_wholefruit', 'hei_totproteins', 'hei_seafoodplantprot',
                    'hei_wholegrains', 'hei_dairy','Fats']:
            if key == 'Fats':
                adeq_check(df, key, values['name'], values['parameters'])
            else:
                adeq_check(df, key, values['name'], values['parameters'][0])
        if key in ['hei_sodium','hei_refinedgrains','hei_addedsugars','hei_SFA']:
            mod_check(df, key, values['name'], values['parameters'])

    df['HEI2015_TOTAL_SCORE']=df[df.columns.intersection(toSum)].sum(axis=1)
    

In [45]:
for key, item in data_dict.items():
    check(para_dict, item)

Calculating score for hei_totveg
Calculating score for hei_greensbeans
Calculating score for hei_totfruit
Calculating score for hei_wholefruit
Calculating score for hei_wholegrains
Calculating score for hei_dairy
Calculating score for hei_totproteins
Calculating score for hei_seafoodplantprot
Calculating score for hei_refinedgrains
Calculating score for hei_addedsugars
Calculating score for hei_SFA
Calculating score for Fats
Calculating score for hei_sodium
     HEIX10_SODIUM  HEIX11_REFINEDGRAIN  HEIX12_ADDEDSUGARS  HEIX13_SATFATS  \
0         3.905811            10.000000                   0         1.24250   
3         2.945969             1.245455                   0         0.57500   
4         0.000000             4.163869                   0         5.70875   
5         2.041536             8.361246                   0         6.60625   
6         4.753373             8.001221                   0         2.02250   
7         5.467568             7.633106                   0     

Calculating score for hei_totveg
Calculating score for hei_greensbeans
Calculating score for hei_totfruit
Calculating score for hei_wholefruit
Calculating score for hei_wholegrains
Calculating score for hei_dairy
Calculating score for hei_totproteins
Calculating score for hei_seafoodplantprot
Calculating score for hei_refinedgrains
Calculating score for hei_addedsugars
Calculating score for hei_SFA
Calculating score for Fats
Calculating score for hei_sodium
                HEIX10_SODIUM  HEIX11_REFINEDGRAIN  HEIX12_ADDEDSUGARS  \
Participant ID                                                           
107008               5.470665            10.000000                   0   
109504               7.049336            10.000000                   0   
116769               6.371106            10.000000                   0   
119470               4.924437            10.000000                   0   
131688               6.601740             6.738347                   0   
133077              

In [37]:
data_dict.keys()

dict_keys(['hei0409', 'dailyhei0409'])

In [47]:
data_dict['hei0409']

,Project Abbreviation,Participant ID,Date of Intake,Project Name,Record Type,Gender,Date of Entry,Day of Intake,Visit Number,Interviewer ID,...,HEIX5_WHOLEGRAIN,HEIX6_TOTALDAIRY,HEIX7_TOTPROT,HEIX8_SEAPLANT_PROT,HEIX11_REFINEDGRAIN,HEIX12_ADDEDSUGARS,HEIX13_SATFATS,HEIX9_FATTYACID,HEIX10_SODIUM,HEI2015_TOTAL_SCORE
0,BCPMomMSTR,1995,06/06/2017,Baby ConnectoMe Project. Mom MASTER. Q4 2017,1,3,12/14/2017,2,2,dpl,...,7.775332,10.000000,5.000000,5.000000,10.000000,0,1.24250,2.761365,3.905811,62.809346
3,BCPMomMSTR,17541,05/30/2017,Baby ConnectoMe Project. Mom MASTER. Q4 2017,1,3,01/02/2018,2,1,cjb,...,0.000000,10.000000,2.244770,5.000000,1.245455,0,0.57500,3.628343,2.945969,35.886039
4,BCPMomMSTR,20784,12/13/2016,Baby ConnectoMe Project. Mom MASTER. Q4 2017,1,3,12/14/2017,2,1,dpl,...,2.726653,3.194342,5.000000,3.744274,4.163869,0,5.70875,2.919592,0.000000,42.234313
5,BCPMomMSTR,20784,03/13/2017,Baby ConnectoMe Project. Mom MASTER. Q4 2017,1,3,12/15/2017,1,2,dpl,...,3.226267,2.159169,5.000000,5.000000,8.361246,0,6.60625,8.919682,2.041536,60.715143
6,BCPMomMSTR,20784,06/22/2017,Baby ConnectoMe Project. Mom MASTER. Q4 2017,1,3,12/15/2017,4,3,dpl,...,6.489917,5.131384,5.000000,5.000000,8.001221,0,2.02250,4.059943,4.753373,56.704361
7,BCPMomMSTR,20784,09/21/2017,Baby ConnectoMe Project. Mom MASTER. Q4 2017,1,3,12/15/2017,4,4,dpl,...,3.178897,5.213773,5.000000,5.000000,7.633106,0,0.00000,2.101562,5.467568,47.308710
11,BCPMomMSTR,48138,07/18/2017,Baby ConnectoMe Project. Mom MASTER. Q4 2017,1,3,01/14/2018,2,1,EI,...,5.076688,3.968252,5.000000,0.000000,10.000000,0,2.79500,1.828125,10.000000,50.209925
12,BCPMomMSTR,52157,02/02/2017,Baby ConnectoMe Project. Mom MASTER. Q4 2017,1,3,01/14/2018,4,1,SCN,...,9.655705,10.000000,1.241869,3.880841,6.335857,0,4.66625,0.000000,6.968860,56.316806
13,BCPMomMSTR,52157,05/03/2017,Baby ConnectoMe Project. Mom MASTER. Q4 2017,1,3,01/14/2018,3,2,SCN,...,10.000000,8.586720,5.000000,5.000000,8.724403,0,0.00000,0.000000,10.000000,60.412113
14,BCPMomMSTR,52157,08/03/2017,Baby ConnectoMe Project. Mom MASTER. Q4 2017,1,3,01/14/2018,4,3,SCN,...,3.876343,10.000000,5.000000,5.000000,10.000000,0,0.00000,0.000000,4.408242,55.537942
